In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 50
import matplotlib.pyplot as pyplot
%matplotlib inline
import seaborn as sns
import sklearn
import pickle

## Load in `.csv` file

In [2]:
df = pd.read_csv('data/extracted/crime-data-from-2010-to-present.csv')

In [3]:
df.head()

DR Number        Date Reported        Date Occurred  Time Occurred  \
0  102005556  2010-01-25T00:00:00  2010-01-22T00:00:00           2300   
1  101822289  2010-11-11T00:00:00  2010-11-10T00:00:00           1800   
2  101105609  2010-01-28T00:00:00  2010-01-27T00:00:00           2230   
3  101620051  2010-11-11T00:00:00  2010-11-07T00:00:00           1600   
4  101910498  2010-04-07T00:00:00  2010-04-07T00:00:00           1600   

   Area ID  Area Name  Reporting District  Crime Code Crime Code Description  \
0       20    Olympic                2071         510       VEHICLE - STOLEN   
1       18  Southeast                1803         510       VEHICLE - STOLEN   
2       11  Northeast                1125         510       VEHICLE - STOLEN   
3       16   Foothill                1641         510       VEHICLE - STOLEN   
4       19    Mission                1902         510       VEHICLE - STOLEN   

  MO Codes  Victim Age Victim Sex Victim Descent  Premise Code  \
0      NaN           0        NaN            NaN         101.0   
1      NaN           0        NaN            NaN         101.0   
2      NaN           0        NaN            NaN         108.0   
3      NaN           0        NaN            NaN         101.0   
4      NaN           0        NaN            NaN         101.0   

  Premise Description  Weapon Used Code Weapon Description Status Code  \
0              STREET               NaN                NaN          IC   
1              STREET               NaN                NaN          IC   
2         PARKING LOT               NaN                NaN          IC   
3              STREET               NaN                NaN          IC   
4              STREET               NaN                NaN          IC   

  Status Description  Crime Code 1  Crime Code 2  Crime Code 3  Crime Code 4  \
0        Invest Cont         510.0           NaN           NaN           NaN   
1        Invest Cont         510.0           NaN           NaN           NaN   
2        Invest Cont         510.0           NaN           NaN           NaN   
3        Invest Cont         510.0           NaN           NaN           NaN   
4        Invest Cont         510.0           NaN           NaN           NaN   

     Address Cross Street                                          Location   
0   VAN NESS         15TH  {'latitude': '34.0454', 'needs_recoding': Fals...  
1       88TH         WALL  {'latitude': '33.9572', 'needs_recoding': Fals...  
2       YORK    AVENUE 51  {'latitude': '34.1211', 'needs_recoding': Fals...  
3  EL DORADO    TRUESDALE  {'latitude': '34.241', 'needs_recoding': False...  
4   GLENOAKS        DRELL  {'latitude': '34.3147', 'needs_recoding': Fals...

In [4]:
df.shape

(1993259, 26)

## Converting `date occured` to time series index

In [5]:
# removing empty time stamp from Date Reported and Date Ocurred
df["Date Reported"] = df["Date Reported"].str.replace('T00:00:00', '')
df["Date Occurred"] = df["Date Occurred"].str.replace('T00:00:00', '')

In [6]:
# setting 'date occured' column as datetime object
df['Date Occurred'] = pd.to_datetime(df['Date Occurred'], format= '%Y-%m-%d')

In [7]:
df = df.set_index('Date Occurred').sort_index()

In [8]:
# checking that it all worked
df.head(3)

DR Number Date Reported  Time Occurred  Area ID Area Name  \
Date Occurred                                                              
2010-01-01     102120693    2010-11-17           1117       21   Topanga   
2010-01-01     100504041    2010-01-02           2130        5    Harbor   
2010-01-01     100504042    2010-01-02           1600        5    Harbor   

               Reporting District  Crime Code  \
Date Occurred                                   
2010-01-01                   2156         354   
2010-01-01                    563         420   
2010-01-01                    519         510   

                                        Crime Code Description   MO Codes  \
Date Occurred                                                               
2010-01-01                                   THEFT OF IDENTITY  0928 1822   
2010-01-01     THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)        NaN   
2010-01-01                                    VEHICLE - STOLEN        NaN   

               Victim Age Victim Sex Victim Descent  Premise Code  \
Date Occurred                                                       
2010-01-01             21          M              H         404.0   
2010-01-01              0        NaN            NaN         104.0   
2010-01-01              0        NaN            NaN         101.0   

              Premise Description  Weapon Used Code Weapon Description  \
Date Occurred                                                            
2010-01-01       DEPARTMENT STORE               NaN                NaN   
2010-01-01               DRIVEWAY               NaN                NaN   
2010-01-01                 STREET               NaN                NaN   

              Status Code Status Description  Crime Code 1  Crime Code 2  \
Date Occurred                                                              
2010-01-01             IC        Invest Cont         354.0           NaN   
2010-01-01             IC        Invest Cont         420.0           NaN   
2010-01-01             IC        Invest Cont         510.0           NaN   

               Crime Code 3  Crime Code 4  \
Date Occurred                               
2010-01-01              NaN           NaN   
2010-01-01              NaN           NaN   
2010-01-01              NaN           NaN   

                                                Address Cross Street  \
Date Occurred                                                          
2010-01-01     21800    VICTORY                      BL          NaN   
2010-01-01       900 W  8TH                          ST          NaN   
2010-01-01      1400 E  O                            ST          NaN   

                                                       Location   
Date Occurred                                                     
2010-01-01     {'latitude': '34.1875', 'needs_recoding': Fals...  
2010-01-01     {'latitude': '33.737', 'needs_recoding': False...  
2010-01-01     {'latitude': '33.7926', 'needs_recoding': Fals...

## Data Cleaning
Now that we have the dataframe converted to a format for time series analysis, we can do basic data cleaning steps.

In [9]:
# checking for null values
df.isna().sum()

DR Number                       0
Date Reported                   0
Time Occurred                   0
Area ID                         0
Area Name                       0
Reporting District              0
Crime Code                      0
Crime Code Description          0
MO Codes                   215872
Victim Age                      0
Victim Sex                 185960
Victim Descent             186006
Premise Code                   50
Premise Description           135
Weapon Used Code          1325641
Weapon Description        1325642
Status Code                     3
Status Description              0
Crime Code 1                    9
Crime Code 2              1863098
Crime Code 3              1990024
Crime Code 4              1993163
Address                         0
Cross Street              1659507
Location                        0
dtype: int64

In [10]:
# dropping columns with a majority of missing values
df = df.drop(axis=1, columns=['MO Codes', 
                              'Weapon Used Code', 
                              'Weapon Description', 
                              'Crime Code 1', # same as feature 'Crime Code', can be easily dropped
                              'Crime Code 2', 
                              'Crime Code 3', 
                              'Crime Code 4', 
                              'Cross Street',
                             'Date Reported'])

In [11]:
victim_demo = ['Victim Age', 'Victim Sex', 'Victim Descent']
geog_area= ['Area ID', 'Area Name']
Crime = ['Crime Code', 'Crime Description']


In [12]:
df.isna().sum()

DR Number                      0
Time Occurred                  0
Area ID                        0
Area Name                      0
Reporting District             0
Crime Code                     0
Crime Code Description         0
Victim Age                     0
Victim Sex                185960
Victim Descent            186006
Premise Code                  50
Premise Description          135
Status Code                    3
Status Description             0
Address                        0
Location                       0
dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1993259 entries, 2010-01-01 to 2019-06-22
Data columns (total 16 columns):
DR Number                 int64
Time Occurred             int64
Area ID                   int64
Area Name                 object
Reporting District        int64
Crime Code                int64
Crime Code Description    object
Victim Age                int64
Victim Sex                object
Victim Descent            object
Premise Code              float64
Premise Description       object
Status Code               object
Status Description        object
Address                   object
Location                  object
dtypes: float64(1), int64(6), object(9)
memory usage: 258.5+ MB


Looks like the 2019 Date reported values have been mistyped and input. May end up dropping this feature.

In [14]:
df['Premise Code'].fillna(0, inplace = True)
df['Premise Description'].fillna('UNKNOWN', inplace = True)
df['Status Code'].fillna('UNK', inplace = True)

In [15]:
df.describe().apply(lambda s: s.apply('{0:.5f}'.format))

DR Number  Time Occurred        Area ID Reporting District  \
count    1993259.00000  1993259.00000  1993259.00000      1993259.00000   
mean   145268085.51049     1361.52226       11.10423         1156.81916   
std     27591418.13650      646.73725        6.00733          600.73388   
min          210.00000        1.00000        1.00000          100.00000   
25%    121204059.50000      930.00000        6.00000          644.00000   
50%    150206637.00000     1430.00000       12.00000         1203.00000   
75%    170710205.00000     1900.00000       16.00000         1672.00000   
max    910220366.00000     2359.00000       21.00000         2199.00000   

          Crime Code     Victim Age   Premise Code  
count  1993259.00000  1993259.00000  1993259.00000  
mean       506.92094       31.78082      311.15603  
std        210.47090       20.60810      211.03603  
min        110.00000       -9.00000        0.00000  
25%        330.00000       20.00000      102.00000  
50%        441.00000       32.00000      210.00000  
75%        626.00000       46.00000      501.00000  
max        956.00000      118.00000      971.00000

In [16]:
df['Victim Sex'].value_counts()

M    920405
F    840397
X     46409
H        70
N        17
-         1
Name: Victim Sex, dtype: int64

In [17]:
df['Victim Sex'].replace(to_replace=['H', '-', 'N'], value='X', inplace = True)
df['Victim Sex'].fillna('X', inplace= True)

In [18]:
df['Victim Sex'].unique()

array(['M', 'X', 'F'], dtype=object)

In [19]:
# df[(df['Victim Age'] < 0) | (df['Victim Age'] >= 80)]
df[df['Victim Age'] < 0]['Victim Age'].count()

465

In [20]:
df['Victim Age'] = np.where((df['Victim Age'] < 0), 0,df['Victim Age'])

In [21]:
df['Victim Age'].isna().sum()

0

In [22]:
df['Victim Descent'].value_counts()

H    685420
W    483035
B    316944
O    191600
X     68234
A     47941
K      8703
F      2358
C       884
I       872
J       352
P       326
U       183
V       150
Z       102
G        79
S        31
D        21
L        15
-         3
Name: Victim Descent, dtype: int64

In [23]:
descent_dict ={'A': 'Other Asian', 'B': 'Black', 'C': 'Chinese',
               'D': 'Cambodian', 'F': 'Filipino', 'G': 'Guamanian',
               'H': 'Latinx', 'I': 'Native', 'J': 'Japanese', 
               'K': 'Korean', 'L': 'Laotian', 'O': 'Other',
               'P': 'Pacific Islander', 'S': 'Samoan', 'U': 'Hawaiian',
               'V': 'Vietnamese', 'W': 'White',  'X': 'Unknown', 'Z':'Asian Indian'}
df['Victim Descent'].fillna('X', inplace= True)

In [24]:
df['Victim Descent'].replace(to_replace=['-'], value='X', inplace = True)

In [25]:
df.isna().sum()

DR Number                 0
Time Occurred             0
Area ID                   0
Area Name                 0
Reporting District        0
Crime Code                0
Crime Code Description    0
Victim Age                0
Victim Sex                0
Victim Descent            0
Premise Code              0
Premise Description       0
Status Code               0
Status Description        0
Address                   0
Location                  0
dtype: int64

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1993259 entries, 2010-01-01 to 2019-06-22
Data columns (total 16 columns):
DR Number                 int64
Time Occurred             int64
Area ID                   int64
Area Name                 object
Reporting District        int64
Crime Code                int64
Crime Code Description    object
Victim Age                int64
Victim Sex                object
Victim Descent            object
Premise Code              float64
Premise Description       object
Status Code               object
Status Description        object
Address                   object
Location                  object
dtypes: float64(1), int64(6), object(9)
memory usage: 258.5+ MB


In [27]:
df[df['Time Occurred'] < 100]

DR Number  Time Occurred  Area ID  Area Name  \
Date Occurred                                                 
2010-01-01     100204011             50        2    Rampart   
2010-01-01     100204018             30        2    Rampart   
2010-01-01     100204019             10        2    Rampart   
2010-01-01     100708906              1        7   Wilshire   
2010-01-01     100708442              1        7   Wilshire   
...                  ...            ...      ...        ...   
2019-06-22     191610620             20       16   Foothill   
2019-06-22     190811604              1        8    West LA   
2019-06-22     191610688             10       16   Foothill   
2019-06-22     191814725             15       18  Southeast   
2019-06-22     190315339             30        3  Southwest   

               Reporting District  Crime Code  \
Date Occurred                                   
2010-01-01                    256         220   
2010-01-01                    249         624   
2010-01-01                    257         740   
2010-01-01                    743         354   
2010-01-01                    745         956   
...                           ...         ...   
2019-06-22                   1613         626   
2019-06-22                    817         354   
2019-06-22                   1659         330   
2019-06-22                   1829         626   
2019-06-22                    361         310   

                                          Crime Code Description  Victim Age  \
Date Occurred                                                                  
2010-01-01                                     ATTEMPTED ROBBERY          16   
2010-01-01                              BATTERY - SIMPLE ASSAULT          29   
2010-01-01     VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...          44   
2010-01-01                                     THEFT OF IDENTITY          68   
2010-01-01               LETTERS, LEWD  -  TELEPHONE CALLS, LEWD          26   
...                                                          ...         ...   
2019-06-22                     INTIMATE PARTNER - SIMPLE ASSAULT          36   
2019-06-22                                     THEFT OF IDENTITY          30   
2019-06-22                                 BURGLARY FROM VEHICLE          21   
2019-06-22                     INTIMATE PARTNER - SIMPLE ASSAULT          33   
2019-06-22                                              BURGLARY          23   

              Victim Sex Victim Descent  Premise Code  \
Date Occurred                                           
2010-01-01             M              H         102.0   
2010-01-01             F              H         203.0   
2010-01-01             F              B         502.0   
2010-01-01             F              A         501.0   
2010-01-01             M              W         502.0   
...                  ...            ...           ...   
2019-06-22             F              H         502.0   
2019-06-22             F              O         101.0   
2019-06-22             F              H         101.0   
2019-06-22             F              B         502.0   
2019-06-22             M              H         707.0   

                                        Premise Description Status Code  \
Date Occurred                                                             
2010-01-01                                         SIDEWALK          AA   
2010-01-01                                   OTHER BUSINESS          IC   
2010-01-01     MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)          IC   
2010-01-01                           SINGLE FAMILY DWELLING          IC   
2010-01-01     MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)          IC   
...                                                     ...         ...   
2019-06-22     MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)          IC   
2019-06-22                                           STREET          IC   
2019-06-22              

In [28]:
df['Crime Code Description'].value_counts().count()

140

In [29]:
df.nunique()

DR Number                 1993259
Time Occurred                1439
Area ID                        21
Area Name                      21
Reporting District           1294
Crime Code                    140
Crime Code Description        140
Victim Age                    101
Victim Sex                      3
Victim Descent                 19
Premise Code                  325
Premise Description           320
Status Code                    10
Status Description              6
Address                     74330
Location                    63249
dtype: int64

In [30]:
def dictionary_maker(key_feature, value_feature):
    '''
    this function takes in two features
    and returns a dictionary with the unique values
    from the two features as key and values
    '''
    # dictionary key
    key= list(df[key_feature].value_counts().index)
    
    #dictionary value
    value= list(df[value_feature].value_counts().index)
    
    #dictionary
    dictionary = {key[i]:value[i] for i in range(len(key))}
    
    return dictionary

# def pickle_dictionary(dict_to_pickle, pickled_dict_name):
#     '''
#     Pickles a dictionary object 
#     '''
#     pickle_out = open(str(pickled_dict_name),"wb")
#     pickle.dump(dict_to_pickle, pickle_out)
#     pickle_out.close()

# pickle.dump( status_dict, open( "status_dict.p", "wb" ) )
# pickle_in = open("dict.pickle","rb")
# example_dict = pickle.load(pickle_in)

In [31]:
area_dict = dictionary_maker('Area ID', 'Area Name')
crime_dict = dictionary_maker('Crime Code', 'Crime Code Description')

In [32]:
df['Premise Description'].isna().any()

False

In [33]:
df['Status Code'].value_counts()

IC     1538964
AO      229649
AA      204515
JA       15193
JO        4907
CC          25
UNK          3
19           1
TH           1
13           1
Name: Status Code, dtype: int64

In [34]:
df['Status Code'].replace(to_replace=['CC', 'TH', '19', '13'], value='UNK', inplace = True)

In [35]:
df['Status Description'].value_counts()

Invest Cont     1538964
Adult Other      229649
Adult Arrest     204515
Juv Arrest        15193
Juv Other          4907
UNK                  31
Name: Status Description, dtype: int64

In [36]:
status_dict = dictionary_maker('Status Code', 'Status Description')
status_dict

{'IC': 'Invest Cont',
 'AO': 'Adult Other',
 'AA': 'Adult Arrest',
 'JA': 'Juv Arrest',
 'JO': 'Juv Other',
 'UNK': 'UNK'}

In [37]:
# pickle_dictionary(area_dict, area_dict1)
# pickle.dump( status_dict, open( "status_dict.p", "wb" ) )

In [38]:
pd.set_option('display.max_rows', None)
# df['Premise Code'].value_counts()
df[df['Premise Code']== 967]

DR Number  Time Occurred  Area ID  Area Name  \
Date Occurred                                                 
2017-06-02     170115839            120        1    Central   
2017-07-03     170118828           1930        1    Central   
2017-10-27     170128107           2315        1    Central   
2017-11-22     170130221           2100        1    Central   
2017-11-30     170130744           1600        1    Central   
2017-12-26     170132653           1705        1    Central   
2018-02-16     180307134           1930        3  Southwest   
2018-02-28     180108707           1730        1    Central   
2018-05-09     180113331            715        1    Central   
2018-07-16     180119129           1200        1    Central   
2018-08-17     180121605           1950        1    Central   
2018-12-03     180129460           1735        1    Central   
2019-02-04     190106657           2200        1    Central   

               Reporting District  Crime Code  \
Date Occurred                                   
2017-06-02                    111         350   
2017-07-03                    111         440   
2017-10-27                    111         624   
2017-11-22                    111         480   
2017-11-30                    111         624   
2017-12-26                    111         624   
2018-02-16                    334         902   
2018-02-28                    111         740   
2018-05-09                    111         860   
2018-07-16                    118         740   
2018-08-17                    111         745   
2018-12-03                    111         624   
2019-02-04                    111         624   

                                          Crime Code Description  Victim Age  \
Date Occurred                                                                  
2017-06-02                                         THEFT, PERSON          26   
2017-07-03                    THEFT PLAIN - PETTY ($950 & UNDER)          37   
2017-10-27                              BATTERY - SIMPLE ASSAULT          41   
2017-11-22                                         BIKE - STOLEN          52   
2017-11-30                              BATTERY - SIMPLE ASSAULT          56   
2017-12-26                              BATTERY - SIMPLE ASSAULT          18   
2018-02-16              VIOLATION OF TEMPORARY RESTRAINING ORDER          51   
2018-02-28     VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...           0   
2018-05-09                           BATTERY WITH SEXUAL CONTACT          48   
2018-07-16     VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...          33   
2018-08-17              VANDALISM - MISDEAMEANOR ($399 OR UNDER)           0   
2018-12-03                              BATTERY - SIMPLE ASSAULT          51   
2019-02-04                              BATTERY - SIMPLE ASSAULT          48   

              Victim Sex Victim Descent  Premise Code  \
Date Occurred                                           
2017-06-02             M              H         967.0   
2017-07-03             M              H         967.0   
2017-10-27             M              H         967.0   
2017-11-22             M              H         967.0   
2017-11-30             M              W         967.0   
2017-12-26             M              H         967.0   
2018-02-16             F              B         967.0   
2018-02-28             M              X         967.0   
2018-05-09             F              H         967.0   
2018-07-16             M              H         967.0   
2018-08-17             X              X         967.0   
2018-12-03             F              W         967.0   
2019-02-04             M              H         967.0   

                       Premise Description Status Code Status Description  \
Date Occurred                                                               
2017-06-02     MTA - GOLD LINE - CHINATOWN          IC        Invest Cont   
2017-07-03     MTA - GOLD LINE - CHINATOWN   

In [39]:
df['Premise Description'].value_counts()
# df[df['Premise Description']=='UNKNOWN']

                                                  2627
LIBRARY                                                              2525
NURSING/CONVALESCENT/RETIREMENT HOME                                 2424
AUTO REPAIR SHOP                                                     2290
STORAGE SHED                                                         2188
COFFEE SHOP (STARBUCKS, COFFEE BEAN, PEET'S, ETC.)                   2110
SPECIALTY SCHOOL/OTHER                                               2084
WAREHOUSE                                                            1932
BEAUTY/BARBER SHOP                                                   1906
GROUP HOME                                                           1772
MTA BUS                                                              1712
DISCOUNT STORE (99 CENT,DOLLAR,ETC.                                  1613
MOBILE HOME/TRAILERS/CONSTRUCTION TRAILERS/RV'S/MOTORHOME            1487
HARDWARE/BUILDING SUPPLY                                 

In [40]:
df['Premise Code'] = np.where(df['Premise Description']== 'UNKNOWN', 0.0, df['Premise Code'])

In [41]:
df.columns

Index(['DR Number', 'Time Occurred', 'Area ID', 'Area Name',
       'Reporting District', 'Crime Code', 'Crime Code Description',
       'Victim Age', 'Victim Sex', 'Victim Descent', 'Premise Code',
       'Premise Description', 'Status Code', 'Status Description', 'Address',
       'Location '],
      dtype='object')

In [42]:
# removing whitespace from column name
df = df.rename(columns={'Location ': 'Location'})

In [43]:
# dictionary_maker('Premise Code', 'Premise Description')

In [44]:
df.columns

Index(['DR Number', 'Time Occurred', 'Area ID', 'Area Name',
       'Reporting District', 'Crime Code', 'Crime Code Description',
       'Victim Age', 'Victim Sex', 'Victim Descent', 'Premise Code',
       'Premise Description', 'Status Code', 'Status Description', 'Address',
       'Location'],
      dtype='object')

####  Location Column Cleaning

In [45]:
# pulling out whole latitude dict
df['latitude'] = df['Location'].str.extract('(\'latitude\'\:\s\'\-?\d+\.?\d+\')', expand=False).str.strip()
# pulling out only coordinate
df['latitude'] = df['latitude'].str.extract('(-?\d+\.?\d+)', expand=False).str.strip()

In [46]:
# pulling out whole latitude dict
df['longitude'] = df['Location'].str.extract('(\'longitude\'\:\s\'\-?\d+\.?\d+\')', expand=False).str.strip()
# pulling out only coordinate
df['longitude'] = df['longitude'].str.extract('(-?\d+\.?\d+)', expand=False).str.strip()

In [47]:
# crime_dict
df.head()

DR Number  Time Occurred  Area ID Area Name  \
Date Occurred                                                
2010-01-01     102120693           1117       21   Topanga   
2010-01-01     100504041           2130        5    Harbor   
2010-01-01     100504042           1600        5    Harbor   
2010-01-01     100504046           1800        5    Harbor   
2010-01-01     100504048           1425        5    Harbor   

               Reporting District  Crime Code  \
Date Occurred                                   
2010-01-01                   2156         354   
2010-01-01                    563         420   
2010-01-01                    519         510   
2010-01-01                    506         330   
2010-01-01                    503         624   

                                        Crime Code Description  Victim Age  \
Date Occurred                                                                
2010-01-01                                   THEFT OF IDENTITY          21   
2010-01-01     THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)           0   
2010-01-01                                    VEHICLE - STOLEN           0   
2010-01-01                               BURGLARY FROM VEHICLE          49   
2010-01-01                            BATTERY - SIMPLE ASSAULT          52   

              Victim Sex Victim Descent  Premise Code     Premise Description  \
Date Occurred                                                                   
2010-01-01             M              H         404.0        DEPARTMENT STORE   
2010-01-01             X              X         104.0                DRIVEWAY   
2010-01-01             X              X         101.0                  STREET   
2010-01-01             M              H         501.0  SINGLE FAMILY DWELLING   
2010-01-01             F              H         101.0                  STREET   

              Status Code Status Description  \
Date Occurred                                  
2010-01-01             IC        Invest Cont   
2010-01-01             IC        Invest Cont   
2010-01-01             IC        Invest Cont   
2010-01-01             IC        Invest Cont   
2010-01-01             AO        Adult Other   

                                                Address  \
Date Occurred                                             
2010-01-01     21800    VICTORY                      BL   
2010-01-01       900 W  8TH                          ST   
2010-01-01      1400 E  O                            ST   
2010-01-01      1500 W  216TH                        ST   
2010-01-01                                        257TH   

                                                        Location latitude  \
Date Occurred                                                               
2010-01-01     {'latitude': '34.1875', 'needs_recoding': Fals...  34.1875   
2010-01-01     {'latitude': '33.737', 'needs_recoding': False...   33.737   
2010-01-01     {'latitude': '33.7926', 'needs_recoding': Fals...  33.7926   
2010-01-01     {'latitude': '33.8325', 'needs_recoding': Fals...  33.8325   
2010-01-01     {'latitude': '33.7909', 'needs_recoding': Fals...  33.7909   

               longitude  
Date Occurred             
2010-01-01      -118.603  
2010-01-01     -118.2967  
2010-01-01     -118.2468  
2010-01-01     -118.3039  
2010-01-01      -118.296

In [48]:
# dropping description columns and features not relevant to modeling
cleaned_features=['DR Number', 'Time Occurred','Area ID', 'Area Name',
                  'Reporting District', 
                  'Crime Code', 'Crime Code Description',
                  'Victim Age',
                  'Victim Sex', 
                  'Victim Descent', 
                  'Premise Code', 'Premise Description',
                  'Status Code', 'Status Description',
                  'latitude', 'longitude']

## Pickling final cleaned df

In [51]:
cleaned_df = df[cleaned_features]

In [52]:
cleaned_df.to_pickle("data/pickled/cleaned_df.pkl")

### To load pickled dataframe
name_for_the_df = pd.read_pickle('data/pickled/cleaned_df.pkl')

Essentially the variables can be grouped to 
* When the crime occurred - given by date and time
* When the crime was reported - Date
* what was the crime - crime code and crime description
* Who was the victim - Victim sex, age and descent
* What weapon was involved - weapon code and description
* Where the crime took place - given by location, address, cross street, premise code and description